This notebook tests linear regression models on the merged data set to predict soil moisture.  

Ridge, Lasso, SVR (RBF/Linear Kernels) and Kernel Ridge Regression (RBF/Linear) are test.

In [1]:
import numpy as np
%matplotlib inline  
import matplotlib.pyplot as plt  
import pandas as pd
import sklearn.linear_model
import sklearn.preprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import time

In [3]:
# Tell the notebook to display all of the results.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Data Import

In [4]:
# ndre_data = pd.read_csv('../data/final_join_subbed_missing_soilM.csv')
# ndre_data_sub = pd.read_csv('../data/final_join_subbed_bare_soil_40.csv')
# ndre_data_lag_40 = pd.read_csv('../data/final_join_subbed_bare_soil_with_lag_40.csv')
ndre_data_lag_40_EXACT = pd.read_csv('../data/final_join_subbed_bare_soil_with_lag_40_EXACT_NDRE.csv')

In [8]:
# ndre_data.columns

In [6]:
ndre_data_lag_40_EXACT.columns

Index(['ndre_val', 'sensor', 'field', 'date', 'data', 'sensor_full_name',
       'depth_1', 'depth_2', 'depth_3', 'year', 'precip.cm', 'tair.C',
       'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'sand_1', 'sand_2',
       'sand_3', 'silt_1', 'silt_2', 'silt_3', 'clay_1', 'clay_2', 'clay_3',
       'l3_depth_1', 'l3_depth_2', 'l3_depth_3', 'l5_depth_1', 'l5_depth_2',
       'l5_depth_3', 'l7_depth_1', 'l7_depth_2', 'l7_depth_3', 'avg_soilM',
       'avg_soilM_l3', 'avg_soilM_l5', 'avg_soilM_l7', 'avg_clay', 'avg_sand',
       'avg_silt'],
      dtype='object')

In [7]:
# keep_cols = ['avg_soilM', 'average_adjacent', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
#              'irradiance.w_per_m.2', 'sand_1', 'sand_2', 'sand_3', 'silt_1', 'silt_2', 'silt_3', 'clay_1', 
#              'clay_2', 'clay_3']
keep_cols = ['avg_soilM', 'ndre_val', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
             'irradiance.w_per_m.2', 'avg_clay', 'avg_sand', 'avg_silt']

In [9]:
dataset = ndre_data_lag_40_EXACT[keep_cols]

In [10]:
dataset.columns

Index(['avg_soilM', 'ndre_val', 'precip.cm', 'tair.C', 'rh.pct',
       'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'avg_clay', 'avg_sand',
       'avg_silt'],
      dtype='object')

In [11]:
X = dataset.iloc[:,1:]
X = X.values

y = dataset.iloc[:,0]
y = y.values

In [149]:
# split data into train and test sets
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

Normalization

In [150]:
mean_X = X_train.mean(axis=0)
std_X = X_train.std(axis=0)

X_train = (X_train - mean_X) / std_X
X_test = (X_test - mean_X) / std_X

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean_y = y_train.mean(axis=0)
std_y = y_train.std(axis=0)

y_train = (y_train - mean_y) / std_y
y_test = (y_test - mean_y) / std_y

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [151]:
print(X_train.shape)
print(X_test.shape)
print('')
print(y_train.shape)
print(y_test.shape)

(299, 7)
(75, 7)

(299,)
(75,)


Ridge model

In [28]:
from sklearn.linear_model import RidgeCV

rcv = RidgeCV(alphas=(0.0001, 0.001, 0.01,0.1, 1.0, 10.0, 100.0, 1000.0), 
              fit_intercept = False, cv=3)
rcv.fit(X_train, y_train)
beta_star = rcv.coef_
rcv.alpha_
beta_star

RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

10.0

array([-0.25743039, -0.05579542, -0.20618273,  0.28996193,  0.16795333,
        0.01071864, -0.05943961,  0.13982845, -0.01182083])

In [27]:
from sklearn import metrics

In [23]:
y_pred_rcv = rcv.predict(X_test)

MAE Ridge

In [24]:
y_pred_inv = (y_pred_rcv * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y

In [25]:
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.03458532650672973

In [54]:
mae_list_rr = []

for i in range(100):
    dataset = ndre_data_lag_40_EXACT[keep_cols]

    X = dataset.iloc[:,1:]
    X = X.values

    y = dataset.iloc[:,0]
    y = y.values

    # split data into train and test sets
    test_size = 0.20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    mean_X = X_train.mean(axis=0)
    std_X = X_train.std(axis=0)

    X_train = (X_train - mean_X) / std_X
    X_test = (X_test - mean_X) / std_X

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    mean_y = y_train.mean(axis=0)
    std_y = y_train.std(axis=0)

    y_train = (y_train - mean_y) / std_y
    y_test = (y_test - mean_y) / std_y

    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')

    
    rcv = RidgeCV(alphas=(0.0001, 0.001, 0.01,0.1, 1.0, 10.0, 100.0, 1000.0), 
                  fit_intercept = False, 
                  cv=3);
   
    rcv.fit(X_train, y_train)
    print('Best Param {0}'.format(rcv.alpha_))
    yhat = rcv.predict(X_test)
    y_pred_inv = (yhat * std_y) + mean_y
    y_test_inv = (y_test * std_y) + mean_y
    mae_list_rr.append(mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv))

RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

Best Param 10.0


RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

Best Param 10.0


RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

...

Best Param 10.0


RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

Best Param 10.0


RidgeCV(alphas=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), cv=3,
    fit_intercept=False, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

Best Param 10.0


In [55]:
np.asarray(mae_list_rr).mean()
np.asarray(mae_list_rr).std()

0.03968124521231422

0.003538487781132812

In [57]:
mae_df_rr = pd.DataFrame({'mae_rr': mae_list_rr})
mae_df_rr.to_csv('../data/results_data/results_df_lin_RR.csv')

Support Vector Machine for Regression

In [15]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

Linear Kernel

Method 1

Linear SVR

In [62]:
svrl = GridSearchCV(SVR(kernel='linear', gamma=0.1), cv=3,
                   param_grid={"C": [1e-4,1e-3,1e-2,1e-1,1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-5, 1, 10)})

In [63]:
t0 = time.time()
svrl.fit(X_train, y_train)
svrl_fit = time.time() - t0
print("SVR complexity and bandwidth selected and model fitted in %.3f s"
      % svrl_fit)

GridSearchCV(cv=3, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'gamma': array([1.00000e-05, 4.64159e-05, 2.15443e-04, 1.00000e-03, 4.64159e-03,
       2.15443e-02, 1.00000e-01, 4.64159e-01, 2.15443e+00, 1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

SVR complexity and bandwidth selected and model fitted in 80.654 s


In [64]:
svrl.best_params_

{'C': 0.1, 'gamma': 1e-05}

In [163]:
mae_list_rr = []

for i in range(100):
    dataset = ndre_data_lag_40[keep_cols]

    X = dataset.iloc[:,1:]
    X = X.values

    y = dataset.iloc[:,0]
    y = y.values

    # split data into train and test sets
    test_size = 0.20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    mean_X = X_train.mean(axis=0)
    std_X = X_train.std(axis=0)

    X_train = (X_train - mean_X) / std_X
    X_test = (X_test - mean_X) / std_X

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    mean_y = y_train.mean(axis=0)
    std_y = y_train.std(axis=0)

    y_train = (y_train - mean_y) / std_y
    y_test = (y_test - mean_y) / std_y

    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    svrl = SVR(kernel='linear', gamma=1e-5, C=0.1, verbose=0)
    svrl.fit(X_train, y_train)

    y_svrl = svrl.predict(X_test)
    y_pred_inv = (y_svrl * std_y) + mean_y
    y_test_inv = (y_test * std_y) + mean_y
    mae_list_rr.append(mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv))

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

...

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=0)

In [164]:
mae_list_rr

[0.03679715618063663,
 0.036857363777694024,
 0.03524190317098836,
 0.03323400030095179,
 0.032934777072607474,
 0.0315043624308765,
 0.0334744954654319,
 0.0345079026676735,
 0.030465640739038933,
 0.03609975759515218,
 0.03541976302395663,
 0.03662232649571337,
 0.03535127370501805,
 0.032620929662273535,
 0.03543619492042765,
 0.029023517994413984,
 0.03225169223033918,
 0.03685106641167217,
 0.030630938285628666,
 0.03481684930783279,
 0.035744757447343324,
 0.03599628925928842,
 0.03424445550116367,
 0.035625015745883774,
 0.032356482444956015,
 0.030840955641766598,
 0.03271281762303323,
 0.032082423577174043,
 0.030963764852447215,
 0.03151473051299424,
 0.030990736499905604,
 0.03397435134734251,
 0.033234272465068514,
 0.030074312326635173,
 0.031778828230716843,
 0.04136352532475826,
 0.03446790525593537,
 0.037418207513417095,
 0.03184221138843802,
 0.03973598312884572,
 0.03210410628715523,
 0.037234645351350314,
 0.032626683365953056,
 0.035758765629593514,
 0.035630415767

In [65]:
t0 = time.time()
y_svrl = svrl.predict(X_test)
svrl_predict = time.time() - t0

In [66]:
y_pred_inv = (y_svrl * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.03525022149291539

Gaussian SVR

In [16]:
svrg = GridSearchCV(SVR(kernel='rbf', gamma=0.1, epsilon = 0.1, C=10), cv=3,
                   param_grid={"C": [1e-4,1e-3,1e-2,1e-1,1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-5, 1, 10),
                              "epsilon": [0.0001,0.001,0.01,0.1,1]})

In [17]:
t0 = time.time()
svrg.fit(X_train, y_train)
svrg_fit = time.time() - t0
print("SVR complexity and bandwidth selected and model fitted in %.3f s"
      % svrg_fit)

GridSearchCV(cv=3, error_score='raise',
       estimator=SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'gamma': array([1.00000e-05, 4.64159e-05, 2.15443e-04, 1.00000e-03, 4.64159e-03,
       2.15443e-02, 1.00000e-01, 4.64159e-01, 2.15443e+00, 1.00000e+01]), 'epsilon': [0.0001, 0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

SVR complexity and bandwidth selected and model fitted in 7.608 s


In [18]:
svrg.best_params_

{'C': 1.0, 'epsilon': 0.1, 'gamma': 0.46415888336127725}

In [19]:
t0 = time.time()
y_svrg = svrg.predict(X_test)
svrg_predict = time.time() - t0

In [20]:
y_pred_inv = (y_svrg * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.03207841236342539

In [12]:
mae_list_svrg = []

for i in range(100):
    dataset = ndre_data_lag_40_EXACT[keep_cols]

    X = dataset.iloc[:,1:]
    X = X.values

    y = dataset.iloc[:,0]
    y = y.values

    # split data into train and test sets
    test_size = 0.20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    mean_X = X_train.mean(axis=0)
    std_X = X_train.std(axis=0)

    X_train = (X_train - mean_X) / std_X
    X_test = (X_test - mean_X) / std_X

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    mean_y = y_train.mean(axis=0)
    std_y = y_train.std(axis=0)

    y_train = (y_train - mean_y) / std_y
    y_test = (y_test - mean_y) / std_y

    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    svrg = SVR(kernel='rbf', gamma=0.46415888336127725, epsilon = 0.1, C=1.0, verbose=False)
    svrg.fit(X_train, y_train)
    y_svrg = svrg.predict(X_test)
    y_pred_inv = (y_svrg * std_y) + mean_y
    y_test_inv = (y_test * std_y) + mean_y
    mae_list_svrg.append(mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv))

NameError: name 'SVR' is not defined

In [187]:
mae_list_svrg

[0.025767921771650146,
 0.02594499662713414,
 0.026220743531916786,
 0.031148760655807125,
 0.024870865663405618,
 0.023797166304509447,
 0.025241901192766383,
 0.02797809621360102,
 0.028460172015283412,
 0.025741731362649928,
 0.025270294697410513,
 0.026787497008990944,
 0.028270010926575124,
 0.02450877047479165,
 0.025274919199629008,
 0.031461522159412184,
 0.0282458865273246,
 0.0261285174532825,
 0.030570171109887904,
 0.029389818358566425,
 0.02588253177534667,
 0.02814531243703278,
 0.028387738020647243,
 0.025560146492469477,
 0.02648314416073228,
 0.025849045420358405,
 0.028532100464734273,
 0.03153839555094229,
 0.026199858317330016,
 0.031436128202455685,
 0.028625696113486025,
 0.03150114993459762,
 0.02965648984824113,
 0.025007771079072563,
 0.028120656000500864,
 0.028461168503668078,
 0.02928445731774536,
 0.03132335702822851,
 0.025201728616911986,
 0.027317793585072348,
 0.030135835438636734,
 0.022005933759341127,
 0.02893306264263675,
 0.02850251472006202,
 0.02

In [188]:
results_df_lin = pd.DataFrame({'rr_mae_l3': mae_list_rr, 'svrg_mae_l3': mae_list_svrg})

In [190]:
results_df_lin.to_csv('../data/results_data/results_df_lin.csv')

#### Kernel Ridge Regression

In [83]:
from sklearn.kernel_ridge import KernelRidge
import time

Gaussian RBF w/ KRR

In [108]:
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e-3, 1e-2, 1e-1, 1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-5, 2, 10)})

In [109]:
t0 = time.time()
kr.fit(X_train, y_train)
kr_fit = time.time() - t0
print("KRR complexity and bandwidth selected and model fitted in %.3f s"
      % kr_fit)

GridSearchCV(cv=5, error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
      kernel_params=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.001, 0.01, 0.1, 1.0, 0.1, 0.01, 0.001], 'gamma': array([1.00000e-05, 5.99484e-05, 3.59381e-04, 2.15443e-03, 1.29155e-02,
       7.74264e-02, 4.64159e-01, 2.78256e+00, 1.66810e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

KRR complexity and bandwidth selected and model fitted in 1.976 s


In [112]:
kr.best_params_

{'alpha': 0.01, 'gamma': 0.01291549665014884}

In [110]:
t0 = time.time()
y_kr = kr.predict(X_test)
kr_predict = time.time() - t0

In [111]:
y_pred_inv = (y_kr * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.028859219

Linear KRR

In [66]:
krl = GridSearchCV(KernelRidge(kernel='linear', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 5)})

In [67]:
t0 = time.time()
krl.fit(X_train, y_train)
krl_fit = time.time() - t0
print("KRR complexity and bandwidth selected and model fitted in %.3f s"
      % krl_fit)

GridSearchCV(cv=5, error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.1, kernel='linear',
      kernel_params=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1.0, 0.1, 0.01, 0.001], 'gamma': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

KRR complexity and bandwidth selected and model fitted in 0.289 s


In [68]:
t0 = time.time()
y_krl = krl.predict(X_test)
krl_predict = time.time() - t0

In [69]:
y_pred_inv = (y_krl * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.036473174

### Lasso model

In [92]:
dataset.columns

Index(['avg_soilM', 'average_adjacent', 'precip.cm', 'tair.C', 'rh.pct',
       'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'avg_clay', 'avg_sand',
       'avg_silt'],
      dtype='object')

In [93]:
from sklearn.linear_model import LassoCV

lcv = LassoCV(alphas=(0.001, 0.01, 0.1, 1.0, 10.0, 100, 1000), fit_intercept = False)
lcv.fit(X_train,y_train)
beta_lcv = lcv.coef_
beta_lcv

LassoCV(alphas=(0.001, 0.01, 0.1, 1.0, 10.0, 100, 1000), copy_X=True, cv=None,
    eps=0.001, fit_intercept=False, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

array([-0.14743337, -0.16570306, -0.18301122,  0.27545294,  0.24043232,
       -0.        ,  0.        ,  0.11334484, -0.05647194], dtype=float32)

In [94]:
y_pred_lcv = lcv.predict(X_test)

In [95]:
lasso_coef = pd.DataFrame()
lasso_coef['feature'] = dataset.columns[1:]
lasso_coef['lasso coef'] = beta_lcv
lasso_coef['ridge coef'] = beta_star
lasso_coef

,feature,lasso coef,ridge coef
0,average_adjacent,-0.147433,-0.151583
1,precip.cm,-0.165703,-0.178700
2,tair.C,-0.183011,-0.188960
3,rh.pct,0.275453,0.278392
4,wind_sp.m_per_s,0.240432,0.239640
5,irradiance.w_per_m.2,-0.000000,-0.010548
6,avg_clay,0.000000,-0.013394
7,avg_sand,0.113345,0.121116
8,avg_silt,-0.056472,-0.078300


MAE Lasso

In [74]:
y_pred_inv = (y_pred_lcv * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y

In [75]:
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.03906586

Subsetting data with features from Lasso

In [76]:
X = dataset.iloc[:,[2,3,4,5,9,11]]
X = X.values

y = dataset.iloc[:,0]
y = y.values

In [77]:
# split data into train and test sets
seed = 7
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

Normalization

In [78]:
mean_X = X_train.mean(axis=0)
std_X = X_train.std(axis=0)

X_train = (X_train - mean_X) / std_X
X_test = (X_test - mean_X) / std_X

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean_y = y_train.mean(axis=0)
std_y = y_train.std(axis=0)

y_train = (y_train - mean_y) / std_y
y_test = (y_test - mean_y) / std_y

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [79]:
print(X_train.shape)
print(X_test.shape)
print('')
print(y_train.shape)
print(y_test.shape)

(317, 6)
(57, 6)

(317,)
(57,)


Ridge model with reduced features (post Lasso)

In [80]:
from sklearn.linear_model import RidgeCV

rcv = RidgeCV(alphas=(0.1, 1.0, 10.0, 100, 1000), fit_intercept = False)
rcv.fit(X_train,y_train)
beta_star = rcv.coef_
beta_star


RidgeCV(alphas=(0.1, 1.0, 10.0, 100, 1000), cv=None, fit_intercept=False,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

array([-0.11989806, -0.20862479,  0.23599008,  0.19058765,  0.18942144,
       -0.11714893])

In [81]:
from sklearn import metrics

In [82]:
y_pred_rcv = rcv.predict(X_test)

MAE Ridge

In [83]:
y_pred_inv = (y_pred_rcv * std_y) + mean_y
y_test_inv = (y_test * std_y) + mean_y

In [84]:
mean_absolute_error(y_pred=y_pred_inv, y_true=y_test_inv)

0.037880783713376824